# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [8]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

s = 0

for sqrt_value in generator:
    s += sqrt_value
#    print(sqrt_value)

print(f'Sum is {s}')


limit = 13
generator = square_root_generator(limit)
for sqrt_value in generator:
    print(sqrt_value)

Sum is 8.382332347441762
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [34]:
import dlt
import duckdb

In [40]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")


In [41]:

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))
print("\n\n\n People table below:")

people_table = conn.sql("SELECT * FROM people").df()
display(people_table)

s = people_table['age'].sum()
print(f'First sum of ages is {s}')



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
└─────────────────────┘




 People table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707836194.0019858,yBXUKPtAx1yADA,None
1,2,Person_2,27,City_A,1707836194.0019858,6I0hCEhN3WLmew,None
2,3,Person_3,28,City_A,1707836194.0019858,xl6nG/ih37Q33Q,None
3,4,Person_4,29,City_A,1707836194.0019858,zdi92bDHxfKfiA,None
4,5,Person_5,30,City_A,1707836194.0019858,I07BBGWLrKHd/Q,None


First sum of ages is 140


In [42]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")


In [44]:
people_table = conn.sql("SELECT * FROM people").df()
display(people_table)

s = people_table['age'].sum()
print(f'Second sum of ages is {s}')


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707836194.0019858,yBXUKPtAx1yADA,None
1,2,Person_2,27,City_A,1707836194.0019858,6I0hCEhN3WLmew,None
2,3,Person_3,28,City_A,1707836194.0019858,xl6nG/ih37Q33Q,None
3,4,Person_4,29,City_A,1707836194.0019858,zdi92bDHxfKfiA,None
4,5,Person_5,30,City_A,1707836194.0019858,I07BBGWLrKHd/Q,None
5,3,Person_3,33,City_B,1707836199.2832432,RYebiPR3JjUlUA,Job_3
6,4,Person_4,34,City_B,1707836199.2832432,sxnCYXSYgrnk7w,Job_4
7,5,Person_5,35,City_B,1707836199.2832432,RN0pBCT/by/7Uw,Job_5
8,6,Person_6,36,City_B,1707836199.2832432,ChqP20JXK9qGfQ,Job_6
9,7,Person_7,37,City_B,1707836199.2832432,6Ds/0pHxSyw0Xg,Job_7


Second sum of ages is 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

In [46]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

generators_pipeline.run(people_1(),
										table_name="people_merged",
										write_disposition="replace",
										primary_key='id')

generators_pipeline.run(people_2(),
										table_name="people_merged",
										write_disposition="merge",
										primary_key='id')




LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x0000024D32027B20>, metrics={'1707836431.3201218': [{'started_at': DateTime(2024, 2, 13, 15, 0, 31, 790124, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 13, 15, 0, 32, 316890, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:///c:\\Users\\Usuario\\OneDrive\\0_Spain\\3_Data Science\\DE_Zoomcamp\\DE_Zoomcamp_Assignments\\workshop 1\\dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='generators', loads_ids=['1707836431.3201218'], load_packages=[LoadPackageInfo(load_id='1707836431.3201218', package_path='C:\\Users\\Usuario\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1707836431.3201218', state='loaded', schema=Schema dlt_ipykernel_launcher at 2530591817984, schema_update={'people_merged': {'name': '

In [47]:

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))
print("\n\n\n People_merged table below:")


people_merged_table = conn.sql("SELECT * FROM people_merged").df()
display(people_merged_table)

s = people_merged_table['age'].sum()
print(f'Sum of ages is {s}')


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_merged       │
└─────────────────────┘




 People_merged table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707836430.432518,p78seN/ObcXTxQ,None
1,2,Person_2,27,City_A,1707836430.432518,A5ekPp8pV8WKEA,None
2,5,Person_5,35,City_B,1707836431.3201218,869l63nsh1JrhQ,Job_5
3,7,Person_7,37,City_B,1707836431.3201218,s9Pen1b2PgD7yA,Job_7
4,8,Person_8,38,City_B,1707836431.3201218,mdqwaguGBBupzg,Job_8
5,3,Person_3,33,City_B,1707836431.3201218,bPVmxTDO4cjbNQ,Job_3
6,6,Person_6,36,City_B,1707836431.3201218,JDoGPBf4QYR70Q,Job_6
7,4,Person_4,34,City_B,1707836431.3201218,9yvUYGGZDa5Rlg,Job_4


Sum of ages is 266
